# Importacion de librerias

In [1]:
import requests
import pandas as pd
import tqdm, sys, os
import numpy as np
import sqlite3
import datetime as dt

pd.options.display.max_columns = 6

# Creamos la BBDD

In [21]:
conn = sqlite3.connect('screener.db') 

# Mini CRUD en SQLite

## Crear una Tabla

In [3]:
q = 'CREATE TABLE IF NOT EXISTS "arg_params" ( \
        "id" INTEGER, \
        "ticker_accion" TEXT, \
        "ticker_opcion" TEXT, \
        "ticker_usa" TEXT, \
        PRIMARY KEY("id" AUTOINCREMENT))'

In [4]:
conn.execute(q)

## Crear Registros

In [5]:
q = 'INSERT INTO arg_params (ticker_accion, ticker_opcion, ticker_usa) VALUES ("GGAL", "GFG", "GGAL")'

In [6]:
conn.execute(q)

In [7]:
conn.commit()

In [8]:
q = 'INSERT INTO arg_params (ticker_accion, ticker_opcion) VALUES ("TGSU2", "TGS")'

In [9]:
conn.execute(q)
conn.commit()

## Leer registros

In [10]:
q = 'SELECT * FROM arg_params'
conn.execute(q).fetchall()

[(1, 'GGAL', 'GFG', 'GGAL'), (2, 'TGSU2', 'TGS', None)]

## Modificar Registros

In [11]:
q = 'UPDATE arg_params SET ticker_usa = "TGS" WHERE id=2 '
conn.execute(q)

In [12]:
q = 'SELECT * FROM arg_params'
conn.execute(q).fetchall()

[(1, 'GGAL', 'GFG', 'GGAL'), (2, 'TGSU2', 'TGS', 'TGS')]

## Agregar en forma masiva

In [13]:
tickers = [('YPFD','YPF','YPF'),('COME','COM',None),('PAM','PAM','PAM')]

In [14]:
for ticker in tickers:
    q = f'INSERT INTO arg_params (ticker_accion, ticker_opcion, ticker_usa)  \
        VALUES ("{ticker[0]}", "{ticker[1]}", "{ticker[2]}")'
    conn.execute(q)

In [15]:
q = 'SELECT * FROM arg_params'
conn.execute(q).fetchall()

[(1, 'GGAL', 'GFG', 'GGAL'),
 (2, 'TGSU2', 'TGS', 'TGS'),
 (3, 'YPFD', 'YPF', 'YPF'),
 (4, 'COME', 'COM', 'None'),
 (5, 'PAM', 'PAM', 'PAM')]

In [16]:
conn.commit()

## Borrar un registro

In [17]:
q = 'DELETE FROM arg_params WHERE id=3'
conn.execute(q)

In [18]:
q = 'SELECT * FROM arg_params'
conn.execute(q).fetchall()

[(1, 'GGAL', 'GFG', 'GGAL'),
 (2, 'TGSU2', 'TGS', 'TGS'),
 (4, 'COME', 'COM', 'None'),
 (5, 'PAM', 'PAM', 'PAM')]

In [19]:
conn.commit()

## Cierre de conexion a la base de datos

In [20]:
conn.close()

# Finviz

## Traemos los datos de la WEB

In [22]:
def scrapear(pags=range(1,3)):
    agents = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
    agents += 'Chrome/50.0.2661.75 Safari/537.36'
    header = {"User-Agent": agents, "X-Requested-With": "XMLHttpRequest"}
    vista_simple = '111'
    vista_full = '152&c='+','.join([str(s) for s in list(range(0,71))])

    df = pd.DataFrame()

    paginas = 1000
    with tqdm.tqdm(total=len(pags), file=sys.stdout) as pbar:
        for pagina in pags:
            pbar.update()

            r = str((pagina)*20 + 1)
            url = f'https://finviz.com/screener.ashx?v={vista_full}&r={r}'
            r = requests.get(url, headers=header)
            tablas = pd.read_html(r.text)

            for tabla in tablas:
                valor = tabla[0].values[0]
                if valor == 'No.':
                    data = tabla

                try:
                    if valor.find('Total: ') == 0:
                        paginas = int(valor[7:valor.find('#')-1])//20 +1
                except:
                    pass

            if pagina == paginas:
                break

            data.columns=data.loc[0]
            data = data.drop(0).drop('No.', axis=1).set_index('Ticker')
            df = pd.concat([df,data])

    return df

In [23]:
finviz = scrapear(pags=range(390))

 98%|██████████████████████████████████████████████████████████████████████████████▌ | 383/390 [02:40<00:02,  2.39it/s]


## Arreglamos datos

### Eliminamos símbolos no numéricos

In [24]:
finviz.replace('-',np.nan, inplace=True)
finviz.replace({'%': ''}, regex=True, inplace=True)

### Arreglamos los numericos hibridos

In [25]:
def aNumerico(df, columnas):

    for columna in columnas:
        count_T, count_B, count_M, count_K = 0,0,0,0
        
        for idx, row in finviz.iterrows():
            try:
                trillones = (row[columna]).find('T') 
                billones = (row[columna]).find('B') 
                millones = (row[columna]).find('M')
                miles = (row[columna]).find('K')

                if  trillones > 0:
                    count_T += 1
                    row[columna] = int(round(float(row[columna][:trillones]),2) * 10**12)
                elif  billones > 0:
                    count_B += 1
                    row[columna] = int(round(float(row[columna][:billones]),2) * 10**9)
                elif  millones > 0:
                    count_M += 1
                    row[columna] = int(round(float(row[columna][:millones]),2) * 10**6)
                elif  miles > 0:
                    count_K += 1
                    row[columna] = int(round(float(row[columna][:miles]),2) * 10**3)
                else:
                    row[columna] = int(row[columna])
                    
            except:
                pass
            
        print(f'Columna: {columna}, T->{count_T}, B->{count_B}, M->{count_M}, K->{count_K}')

In [26]:
columnas = ['Market Cap', 'Avg Volume','Outstanding','Float',]

aNumerico(finviz, columnas)

Columna: Market Cap, T->0, B->2337, M->3062, K->0
Columna: Avg Volume, T->0, B->0, M->1699, K->5923
Columna: Outstanding, T->0, B->189, M->5212, K->0
Columna: Float, T->0, B->149, M->4690, K->0


### Pasamos a Flotante lo que sea flotante

In [27]:
def aFlotante(df, columnas):
    for columna in columnas:
        try:
            df[columna] = df[columna].astype(float)
        except:
            try:
                df[columna] = pd.to_numeric(df[columna])
            except:
                print (f'No se pudo trnasformar la columna {columna}')

In [28]:
columnas_no_float = ['Ticker', 'Company', 'Sector', 'Industry', 'Country', 'Market Cap',
                     'Outstanding','Float','Avg Volume','Volume', 'Earnings', 'IPO Date']

columnas_float = [col for col in list(finviz.columns) if col not in columnas_no_float]
aFlotante(finviz,columnas_float)

### Pasamos a Entero lo que es entero

In [29]:
def aEntero(df, columnas):
    for columna in columnas:
        df[columna] = pd.to_numeric(df[columna])

In [30]:
columnas_int = ['Market Cap', 'Outstanding','Float','Avg Volume','Volume']
aEntero(finviz, columnas_int)

### Pasamos a Fecha lo que se pueda pasar a fecha

In [31]:
finviz['IPO Date'] = pd.to_datetime(finviz['IPO Date'], format='%m/%d/%Y')

### Arreglamos nombres de columnas

In [32]:
columnas_nombres = [c.replace(' ','_') for c in list(finviz.columns)]
finviz.columns = columnas_nombres

In [33]:
columnas_nombres = [c.replace('/','_') for c in list(finviz.columns)]
finviz.columns = columnas_nombres

In [34]:
finviz.index.name = 'Symbol'

## Guardamos la Tabla

In [35]:
finviz.to_sql(con=conn, name='finviz', if_exists='replace')

## Levantamos la tabla

In [36]:
%%timeit
q = 'SELECT * FROM finviz'
pd.read_sql(q, conn)

150 ms ± 6.08 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
q = 'SELECT * FROM finviz'
pd.read_sql(q, conn)

,Symbol,Company,Sector,...,Earnings,Target_Price,IPO_Date
0,A,"Agilent Technologies, Inc.",Healthcare,...,Aug 18/a,103.64,1999-11-18 00:00:00
1,AA,Alcoa Corporation,Basic Materials,...,Oct 14/a,14.32,2016-10-18 00:00:00
2,AAAU,Perth Mint Physical Gold ETF,Financial,...,None,NaN,2018-08-15 00:00:00
3,AACG,ATA Creativity Global,Consumer Defensive,...,Aug 12/a,NaN,2008-01-29 00:00:00
4,AACQ,Artius Acquisition Inc.,Financial,...,None,NaN,2020-09-04 00:00:00
...,...,...,...,...,...,...,...
7617,ZUO,"Zuora, Inc.",Technology,...,Sep 02/a,13.58,2018-04-12 00:00:00
7618,ZVO,Zovio Inc,Consumer Defensive,...,Oct 28/a,NaN,2009-04-15 00:00:00
7619,ZYME,Zymeworks Inc.,Healthcare,...,None,24.88,2017-04-28 00:00:00
7620,ZYNE,"Zynerba Pharmaceuticals, Inc.",Healthcare,...,None,NaN,2015-08-05 00:00:00


## Consultas fuera del método de pandas

In [38]:
%%timeit
q = 'SELECT * FROM finviz'
data = conn.execute(q).fetchall()

126 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
q = 'SELECT * FROM finviz'
data = conn.execute(q).fetchall()
data[0][:10]

('A',
 'Agilent Technologies, Inc.',
 'Healthcare',
 'Diagnostics & Research',
 'USA',
 32880000000.0,
 47.92,
 28.9,
 5.1,
 6.29)

## Consultas de INFO de la tabla

In [40]:
q  = 'PRAGMA TABLE_INFO(finviz)'
columnas = conn.execute(q).fetchall()
columnas[:10]

[(0, 'Symbol', 'TEXT', 0, None, 0),
 (1, 'Company', 'TEXT', 0, None, 0),
 (2, 'Sector', 'TEXT', 0, None, 0),
 (3, 'Industry', 'TEXT', 0, None, 0),
 (4, 'Country', 'TEXT', 0, None, 0),
 (5, 'Market_Cap', 'REAL', 0, None, 0),
 (6, 'P_E', 'REAL', 0, None, 0),
 (7, 'Fwd_P_E', 'REAL', 0, None, 0),
 (8, 'PEG', 'REAL', 0, None, 0),
 (9, 'P_S', 'REAL', 0, None, 0)]

* column_name
* type (the datatype, like integer or text),
* notnull (1 if the column has a NOT NULL constraint)
* dflt_value (NULL if no default value)
* pk (1 if the column is the table's primary key, else 0)

## "Desempaquetado" de listas

In [47]:
ids, cols, tipo, nn, dflt, pk = zip(*columnas)

In [49]:
cols[:10]

('Symbol',
 'Company',
 'Sector',
 'Industry',
 'Country',
 'Market_Cap',
 'P_E',
 'Fwd_P_E',
 'PEG',
 'P_S')

# FMP

In [50]:
apikey = '66c44b016256f8b9e507838f32434ece'
apikey = '7bcec62b64a5ccd1f35fc2036c768593'
apikey = '2f08561fe33be29429d696aed288539e'
apikey = '938c4b4dece598e20a115db9b9d3d5e9'
apikey = 'f0bdfc9adbb9cad589dadedfd786d499'

In [51]:
def getPanel(exchange):
    url = 'https://fmpcloud.io/api/v3/quotes/'+exchange
    p = {'apikey': apikey}
    r = requests.get(url, params = p)
    js = r.json()
    df = pd.DataFrame(js)
    df.set_index('symbol', inplace=True)
    return df

## Bajamos la data

In [52]:
paneles = ['etf','commodity','euronext','nyse','amex','nasdaq','tsx','index','mutual_fund']

data_fmp = pd.DataFrame()
for panel in paneles:
    print(panel, end=', ')
    df = getPanel(panel)
    df['panel'] = panel
    data_fmp = pd.concat([data_fmp, df])

etf, commodity, euronext, nyse, amex, nasdaq, tsx, index, mutual_fund, 

In [53]:
data_fmp

,name,price,changesPercentage,...,sharesOutstanding,timestamp,panel
symbol,,,,,,,
EAOA,iShares ESG Aware Aggressive Allocation ETF,26.9300,-2.36,...,NaN,1603914817,etf
WIL,Barclays Women in Leadership ETN,72.3813,0.00,...,NaN,1603914817,etf
NULC,Nuveen ESG Large-Cap ETF,32.0875,0.00,...,NaN,1603914817,etf
HYDB,iShares Edge High Yield Defensive Bond ETF,48.9644,-0.50,...,NaN,1603914817,etf
GOVZ,iShares Trust - iShares 25+ Year Treasury Stri...,24.1900,0.00,...,NaN,1603914817,etf
...,...,...,...,...,...,...,...
NCTWX,Nicholas II Fund Class I,29.0600,-0.82,...,NaN,1603914826,mutual_fund
FHCCX,Fidelity Advisor Health Care Fund Class C,49.5200,-0.30,...,NaN,1603914826,mutual_fund
FSIMX,Salient Select Income Advisor,23.5700,0.13,...,NaN,1603914826,mutual_fund


## Verificamos tipos de datos

In [54]:
data_fmp.dtypes

name                     object
price                   float64
changesPercentage       float64
change                  float64
dayLow                  float64
dayHigh                 float64
yearHigh                float64
yearLow                 float64
marketCap                object
priceAvg50              float64
priceAvg200             float64
volume                  float64
avgVolume               float64
exchange                 object
open                    float64
previousClose           float64
eps                      object
pe                       object
earningsAnnouncement     object
sharesOutstanding        object
timestamp                 int64
panel                    object
dtype: object

## Transformamos a numérico

In [55]:
aFlotante(data_fmp, ['marketCap','eps','pe','earningsAnnouncement','sharesOutstanding'])

No se pudo trnasformar la columna earningsAnnouncement


## Transformamos fechas

In [56]:
data_fmp['timestamp'] = pd.to_datetime(data_fmp['timestamp'], unit='s')

In [57]:
data_fmp

,name,price,changesPercentage,...,sharesOutstanding,timestamp,panel
symbol,,,,,,,
EAOA,iShares ESG Aware Aggressive Allocation ETF,26.9300,-2.36,...,NaN,2020-10-28 19:53:37,etf
WIL,Barclays Women in Leadership ETN,72.3813,0.00,...,NaN,2020-10-28 19:53:37,etf
NULC,Nuveen ESG Large-Cap ETF,32.0875,0.00,...,NaN,2020-10-28 19:53:37,etf
HYDB,iShares Edge High Yield Defensive Bond ETF,48.9644,-0.50,...,NaN,2020-10-28 19:53:37,etf
GOVZ,iShares Trust - iShares 25+ Year Treasury Stri...,24.1900,0.00,...,NaN,2020-10-28 19:53:37,etf
...,...,...,...,...,...,...,...
NCTWX,Nicholas II Fund Class I,29.0600,-0.82,...,NaN,2020-10-28 19:53:46,mutual_fund
FHCCX,Fidelity Advisor Health Care Fund Class C,49.5200,-0.30,...,NaN,2020-10-28 19:53:46,mutual_fund
FSIMX,Salient Select Income Advisor,23.5700,0.13,...,NaN,2020-10-28 19:53:46,mutual_fund


## Guardamos la Tabla

In [58]:
data_fmp.to_sql(con=conn, name='fmp', if_exists='replace')

## Levantamos la tabla

In [59]:
q = 'SELECT * FROM fmp'
pd.read_sql(q, conn)

,symbol,name,price,...,sharesOutstanding,timestamp,panel
0,EAOA,iShares ESG Aware Aggressive Allocation ETF,26.9300,...,NaN,2020-10-28 19:53:37,etf
1,WIL,Barclays Women in Leadership ETN,72.3813,...,NaN,2020-10-28 19:53:37,etf
2,NULC,Nuveen ESG Large-Cap ETF,32.0875,...,NaN,2020-10-28 19:53:37,etf
3,HYDB,iShares Edge High Yield Defensive Bond ETF,48.9644,...,NaN,2020-10-28 19:53:37,etf
4,GOVZ,iShares Trust - iShares 25+ Year Treasury Stri...,24.1900,...,NaN,2020-10-28 19:53:37,etf
...,...,...,...,...,...,...,...
14802,NCTWX,Nicholas II Fund Class I,29.0600,...,NaN,2020-10-28 19:53:46,mutual_fund
14803,FHCCX,Fidelity Advisor Health Care Fund Class C,49.5200,...,NaN,2020-10-28 19:53:46,mutual_fund
14804,FSIMX,Salient Select Income Advisor,23.5700,...,NaN,2020-10-28 19:53:46,mutual_fund
14805,RYHOX,Rydex NASDAQ-100 Fund Class H,53.6500,...,NaN,2020-10-28 19:53:46,mutual_fund


# Alpaca

## Ver que activos tenemos en otras tablas

### Activos de finviz

In [60]:
q = 'SELECT COUNT(*) FROM finviz WHERE Country="USA"'
conn.execute(q).fetchone()

(6633,)

In [61]:
q = 'SELECT symbol FROM finviz WHERE Country="USA"'
symbols_finviz = pd.read_sql(q,conn).Symbol.to_list()
symbols_finviz[:10]

['A', 'AA', 'AAAU', 'AACQ', 'AADR', 'AAL', 'AAMC', 'AAME', 'AAN', 'AAOI']

### Activos de FMP

In [62]:
q = 'SELECT exchange, COUNT(*) as q FROM fmp GROUP BY exchange'
conn.execute(q).fetchall()

[('AMEX', 284),
 ('COMMODITY', 29),
 ('ETF', 487),
 ('EURONEXT', 1251),
 ('INDEX', 61),
 ('MUTUAL_FUND', 2471),
 ('NASDAQ', 4237),
 ('NYSE', 4565),
 ('TSX', 1422)]

In [63]:
q  = 'SELECT symbol FROM fmp WHERE exchange IN ("ETF","AMEX","NYSE","NASDAQ") ORDER BY symbol ASC'
symbols_fmp = pd.read_sql(q,conn).symbol.to_list()
len(symbols_fmp), symbols_fmp[:10]

(9573,
 ['A', 'AA', 'AAA', 'AAAU', 'AABA', 'AAC', 'AACG', 'AACQ', 'AACQU', 'AACQW'])

## Traer Activos posibles

In [64]:
api_key = "PKPH8PLW39GAMTFTFCQQ"
secret_key = "G3FxXZquKtcZhYSQNdsbhlMpJQYDks3R5LwYw7n0"

In [65]:
def assets(only_shorteables=False):    
    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    r = requests.get(url = "https://paper-api.alpaca.markets/v2/assets", 
                      headers = headers)

    js = r.json()
    df = pd.DataFrame(js)
    df = df.loc[df.status=='active'].drop(['status','class','easy_to_borrow',
                                          'tradable','marginable'], axis=1)
    
    if only_shorteables:
        df= df.loc[df.shortable==True]
        
    return df

In [66]:
activos = assets(only_shorteables=False)
tickers = activos.symbol.to_list()

In [67]:
len(tickers)

8965

In [68]:
tickers_ok = [t for t in tickers if t in symbols_finviz]
len(tickers_ok)

6612

In [69]:
def listasActivos():    
    activos = assets(only_shorteables=False)
    tickers = activos.symbol.to_list()
    tickers_ok = [t for t in tickers if t in symbols_finviz]

    listas = []
    for i in range(len(tickers_ok)//200 + 1):
        listas.append(tickers_ok[i*200 : (i+1)*200])

    return listas

listas = listasActivos()

print('\n Ultima lista:\n',listas[len(listas)-1])
print('\n Cantidad de Ultima lista:',len(listas[len(listas)-1]))
print('\n Cantidad de listas de 200:',len(listas)-1)


 Ultima lista:
 ['AIT', 'XLG', 'XLK', 'XLP', 'ZS', 'ZSAN', 'ZSL', 'ZTR', 'ZTS', 'ZUMZ', 'ZUO', 'ZVO']

 Cantidad de Ultima lista: 12

 Cantidad de listas de 200: 33


In [70]:
def sublist_to_string(listas):
    for i in range(len(listas)):
        listas[i] = ','.join(listas[i])
    return listas

listas = sublist_to_string(listasActivos())
listas[0]

'A,AACQ,AADR,AAMC,AAN,AAOI,AAT,AAXJ,AB,ABC,ABEO,ABM,ABR,AAME,AAON,AAP,AAWW,AAXN,ABBV,ABCB,ABEQ,ABG,ABIO,ABMD,ABT,ABUS,ABTX,AC,ACA,ASB,ACAD,ACAM,BIIB,ACBI,ACC,BLES,ACCD,ACCO,ACEL,ACER,ACES,BOX,CE,IDA,IUSV,JMP,JPM,LRGF,MATX,MGK,MSCI,NJR,ACET,ACEV,OMC,PD,PSQ,PVH,AE,SMM,AZZ,BDX,ACEVU,CLNC,DCRBU,FDTS,FLTB,ACHC,ACI,HPQ,IT,MRKR,MVO,ACLS,PCGU,ACNB,DHIL,GGG,TSN,GSIT,ACIA,ACIO,ACIW,MUR,NCR,ACM,ACMR,PKBK,ACOR,ACRE,QID,ACP,ACRS,ACRX,ACSI,ACSG,RES,ACT,ACTG,ACV,ACWF,ACWV,ADBE,U,ADI,ADIL,VER,VNCE,ADMP,ADMS,ADRE,ADSK,ADT,ADUS,ADX,ADXS,AEE,AEGN,AEIS,AEO,AES,AESE,AEY,AEZS,RNG,SBR,ACU,SM,ACWI,ADC,ADES,USL,ADM,ADMA,ADME,WEX,ADPT,ADS,ADSW,ADTN,ADTX,ADVM,AEHR,AEL,AEMD,AEP,AERI,AESR,AEYE,AFB,AFG,AFIB,AFI,AFIF,AFL,AGE,AGFS,AGG,AGGY,AGIO,AGLE,AGNC,AGR,AGRX,AGT,AGX,AGYS,AGZD,AHC,AHH,AHT,AFIN,AFK,AFLG,AFT,AGCO,AGD,AGEN,AGGP,AGM,AGQ,AGS,AGTC,AGZ,AHACU,AHCO,AHPI,AIF,AIA,AIEQ,AIG,AIIQ,AIMC,AINC,AIO,AIR,AIRI,AKUS,ALBO,ALCO'

## Traer historicos con Batch Request

In [71]:
def getHistory(symbol, start, end, timeframe='1D'):    
    start = dt.datetime.strftime(start, format='%Y-%m-%dT%T-04:00')
    end = dt.datetime.strftime(end, format='%Y-%m-%dT%T-04:00')

    headers ={"APCA-API-KEY-ID" : api_key, "APCA-API-SECRET-KEY":secret_key}
    params =  {'symbols' : symbol, 'start':start, 'end':end}
    endpoint = "https://data.alpaca.markets/v1/bars/"+timeframe
    r = requests.get(url = endpoint, headers =headers, params=params)
    js = r.json()
    
    tickers = symbol.split(",")
    for ticker in tickers:
        for i in range(len(js[ticker])):
            js[ticker][i]['ticker']=ticker
    return js

## Pre-Proceso para armado eficiente de DF

### Modelado del problema

In [72]:
orig = {'cl1' : [{'scl1':1,'scl2':2}, {'scl1':3,'scl2':4}],
        'cl2' : [{'scl1':5,'scl2':6}, {'scl1':7,'scl2':8}]}

In [73]:
orig.values()

dict_values([[{'scl1': 1, 'scl2': 2}, {'scl1': 3, 'scl2': 4}], [{'scl1': 5, 'scl2': 6}, {'scl1': 7, 'scl2': 8}]])

In [74]:
[dicc for lista in orig.values() for dicc in lista]

[{'scl1': 1, 'scl2': 2},
 {'scl1': 3, 'scl2': 4},
 {'scl1': 5, 'scl2': 6},
 {'scl1': 7, 'scl2': 8}]

In [75]:
final = []
for lista in orig.values():
    for dicc in lista:
        final.append(dicc)
final

[{'scl1': 1, 'scl2': 2},
 {'scl1': 3, 'scl2': 4},
 {'scl1': 5, 'scl2': 6},
 {'scl1': 7, 'scl2': 8}]

### En el DataFrame Real

In [76]:
%%time
end =  dt.date.today() + dt.timedelta(days=1)
start = end - dt.timedelta(days=2000)

data_js = getHistory(listas[0], start, end, timeframe='1D')

Wall time: 7.49 s


In [77]:
data_js_stack = ([dicc for lista in data_js.values() for dicc in lista])

In [78]:
pd.DataFrame(data_js_stack)

,t,o,h,...,c,v,ticker
0,1431316800,42.39,42.8300,...,42.61,831230,A
1,1431403200,42.36,42.5000,...,41.92,1981361,A
2,1431489600,42.05,42.3900,...,41.80,1781827,A
3,1431576000,42.18,42.2251,...,42.04,2605877,A
4,1431662400,42.14,42.2600,...,42.05,1953869,A
...,...,...,...,...,...,...,...
230861,1603339200,144.81,148.9300,...,148.27,768700,WEX
230862,1603425600,149.99,150.0400,...,146.49,600985,WEX
230863,1603684800,144.48,144.4800,...,140.05,720440,WEX
230864,1603771200,140.39,142.2200,...,138.20,681794,WEX


## Guardamos todo en BBDD

In [79]:
%%time
with tqdm.tqdm(total=len(listas), file=sys.stdout) as pbar:
    for i in range(len(listas)):
        pbar.update()
        end =  dt.date.today() + dt.timedelta(days=1)
        start = end - dt.timedelta(days=1000)
        data_js = getHistory(listas[i], start, end, timeframe='1D')
        data_js_stack = ([dicc for lista in data_js.values() for dicc in lista])    
        df = pd.DataFrame(data_js_stack) 
        df.t = pd.to_datetime(df.t, unit='s')
        df.columns = ['Date','Open','High','Low','Close','Volume','Ticker']
        df = df.loc[df.Close!=0]
        df['pctChange'] = np.where(df.Ticker==df.Ticker.shift(),df.Close.pct_change().round(4)*100,np.nan)
        df.dropna(inplace=True)
        df.to_sql(con=conn, name='alpaca', if_exists='append')

100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [04:18<00:00,  7.61s/it]
Wall time: 4min 18s


## Levantamos la tabla

In [80]:
%%time
q = 'SELECT * FROM alpaca'
df = pd.read_sql(q, conn).set_index('Date')
df

Wall time: 13.1 s


,index,Open,High,...,Volume,Ticker,pctChange
Date,,,,,,,
2018-02-02 05:00:00,1,72.32,72.7600,...,1507746,A,-2.20
2018-02-05 05:00:00,2,70.86,71.4700,...,2147444,A,-4.25
2018-02-06 05:00:00,3,66.96,68.8300,...,3165907,A,0.37
2018-02-07 05:00:00,4,68.19,69.0850,...,1528645,A,-0.60
2018-02-08 05:00:00,5,68.09,68.1600,...,2626815,A,-4.42
...,...,...,...,...,...,...,...
2020-10-22 04:00:00,7889,4.15,4.2400,...,346624,ZVO,0.72
2020-10-23 04:00:00,7890,4.20,4.2400,...,250236,ZVO,0.24
2020-10-26 04:00:00,7891,4.25,4.3307,...,251276,ZVO,-0.72


## Eliminamos datos sospechosos de error por splits etc

In [81]:
%%time
q = 'SELECT DISTINCT Ticker FROM alpaca WHERE pctChange > 200 OR pctChange < -75'
quitar = pd.read_sql(q, conn).Ticker.to_list()
len(quitar), quitar[:10]

Wall time: 689 ms


(316,
 ['ABIO',
  'ACER',
  'ACET',
  'ADXS',
  'AEMD',
  'AGRX',
  'AHPI',
  'AHT',
  'AINV',
  'ALIM'])

In [82]:
quitar_str='"'+'","'.join(quitar)+'"'
quitar_str

'"ABIO","ACER","ACET","ADXS","AEMD","AGRX","AHPI","AHT","AINV","ALIM","ALT","ALTA","ALTM","AMLP","AMRH","AMZA","APDN","APEX","APRE","APRN","APVO","ARTL","ATNM","ATOS","AVEO","AVGR","AXAS","BAR","BCDA","BIOC","BIOL","BIS","BIVI","BLCM","BLIN","BLPH","BMRA","BNGO","BNTC","BOIL","BPTH","BRZU","BW","BZQ","CAPR","CARV","CATB","CBAY","CBLI","CCXI","CEI","CEM","CEN","CETX","CFBK","CFRX","CGIX","CHDN","CHFS","CIDM","CLDX","CLSK","CPE","CPST","CPTA","CRBP","CREX","CRIS","CRMD","CTIB","CTR","CUZ","CYCC","CYCN","CZR","DCTH","DDM","DFFN","DIG","DNB","DOG","DPST","DPW","DRAD","DRIP","DRV","DSS","DUOT","DUST","DX","DXD","EBIX","EDZ","EEV","EFO","EFOI","EGLE","EKSO","EMO","ERX","EQ","ERY","EYEG","EYES","FAZ","FCEL","FLUX","FMO","FNGD","FRAK","FRAN","FSK","FTSI","FVE","GER","GEVO","GNCA","GNPX","GOVX","GPMT","GREK","GRPN","XBIO","GUSH","GXG","HGBL","HGEN","HIBL","HOV","HSON","HTZ","HUSA","ICD","ICON","IDXG","IGC","IJK","IMAC","IMBI","INPX","INUV","IPI","IPWR","IRET","IVW","IWR","JAGX","JAKK","JDST","J

In [83]:
q = f'DELETE FROM alpaca WHERE ticker IN ({quitar_str})'
conn.execute(q)

In [84]:
conn.commit()

# JOINS

## Solo finviz

In [85]:
q = 'SELECT * FROM finviz WHERE finviz.symbol="AAPL"'
pd.read_sql(q, conn)

,Symbol,Company,Sector,...,Earnings,Target_Price,IPO_Date
0,AAPL,Apple Inc.,Technology,...,Oct 29/a,121.43,1980-12-12 00:00:00


## Solo FMP

In [86]:
q = 'SELECT * FROM fmp WHERE fmp.symbol="AAPL"'
pd.read_sql(q, conn)

,symbol,name,price,...,sharesOutstanding,timestamp,panel
0,AAPL,Apple Inc.,112.09,...,1.710250e+10,2020-10-28 19:53:43,nasdaq


## JOIN finviz + FMP

In [87]:
q = 'SELECT * FROM finviz JOIN fmp ON finviz.symbol=fmp.symbol WHERE finviz.symbol="AAPL"'
pd.read_sql(q, conn)

,Symbol,Company,Sector,...,sharesOutstanding,timestamp,panel
0,AAPL,Apple Inc.,Technology,...,1.710250e+10,2020-10-28 19:53:43,nasdaq


### Con alias de tablas

In [88]:
q = 'SELECT * FROM finviz f JOIN fmp fm ON f.symbol=fm.symbol WHERE f.symbol="AAPL"'
pd.read_sql(q, conn)

,Symbol,Company,Sector,...,sharesOutstanding,timestamp,panel
0,AAPL,Apple Inc.,Technology,...,1.710250e+10,2020-10-28 19:53:43,nasdaq


## JOIN Alpaca + Finviz

In [89]:
q = 'SELECT  finviz.* , MAX(alpaca.pctChange) AS max_variacion FROM alpaca \
        JOIN finviz ON finviz.symbol = alpaca.ticker \
        WHERE finviz.symbol="AAPL"'
pd.read_sql(q, conn)

,Symbol,Company,Sector,...,Target_Price,IPO_Date,max_variacion
0,AAPL,Apple Inc.,Technology,...,121.43,1980-12-12 00:00:00,12.12


## Sub-Selects o Queries anidadas

In [90]:
q = '''SELECT * FROM fmp 
        WHERE fmp.pe < 30 AND fmp.pe > 10 AND fmp.symbol IN (
        SELECT symbol FROM finviz WHERE finviz.P_E<30 AND finviz.P_E > 10
        )
        '''
pd.read_sql(q, conn)

,symbol,name,price,...,sharesOutstanding,timestamp,panel
0,AAP,"Advance Auto Parts, Inc.",147.050,...,6.913880e+07,2020-10-28 19:53:40,nyse
1,AB,AllianceBernstein Holding L.P.,29.720,...,9.617460e+07,2020-10-28 19:53:40,nyse
2,ABBV,AbbVie Inc.,80.870,...,1.764830e+09,2020-10-28 19:53:40,nyse
3,ABC,AmerisourceBergen Corporation,96.300,...,2.041420e+08,2020-10-28 19:53:40,nyse
4,ABEV,Ambev S.A.,2.345,...,1.563400e+10,2020-10-28 19:53:40,nyse
...,...,...,...,...,...,...,...
1034,XOM,Exxon Mobil Corporation,31.650,...,4.228230e+09,2020-10-28 19:53:40,nyse
1035,YUM,"Yum! Brands, Inc.",95.430,...,3.014000e+08,2020-10-28 19:53:40,nyse
1036,ZEAL,Zealand Pharma A/S,33.610,...,3.984010e+07,2020-10-28 19:53:43,nasdaq
1037,ZION,"Zions Bancorporation, National Association",30.640,...,1.640090e+08,2020-10-28 19:53:43,nasdaq


# Vistas

* Privacidad
* Optimizacion
* Practicidad

In [91]:
%%time
q = '''CREATE VIEW alpaca_30d AS
        SELECT * FROM alpaca WHERE Date > DateTime("Now", "LocalTime", "-30 Day")
    '''
conn.execute(q)

Wall time: 12 ms


In [92]:
%%time
q = 'SELECT * FROM alpaca_30d'
pd.read_sql(q, conn)

Wall time: 1.05 s


,index,Date,Open,...,Volume,Ticker,pctChange
0,668,2020-09-29 04:00:00,99.79,...,1060273,A,0.66
1,669,2020-09-30 04:00:00,100.54,...,1615853,A,0.68
2,670,2020-10-01 04:00:00,101.77,...,973165,A,0.33
3,671,2020-10-02 04:00:00,100.21,...,1020938,A,-1.20
4,672,2020-10-05 04:00:00,100.99,...,1240651,A,3.08
...,...,...,...,...,...,...,...
133799,7889,2020-10-22 04:00:00,4.15,...,346624,ZVO,0.72
133800,7890,2020-10-23 04:00:00,4.20,...,250236,ZVO,0.24
133801,7891,2020-10-26 04:00:00,4.25,...,251276,ZVO,-0.72
133802,7892,2020-10-27 04:00:00,4.21,...,528293,ZVO,-3.12


## Anido select de tabla con vista

* alpaca_30d es una vista con los ultimos 30 dias de los historicos
* fmp es una tabla con una sola fila de cada ticker

In [93]:
%%time
q = 'SELECT * FROM alpaca_30d WHERE Ticker IN (SELECT symbol FROM fmp WHERE pe < 30 AND pe >10)'
pd.read_sql(q, conn)

Wall time: 768 ms


,index,Date,Open,...,Volume,Ticker,pctChange
0,5432,2020-09-29 04:00:00,153.89,...,794986,AAP,-1.06
1,5433,2020-09-30 04:00:00,151.48,...,814183,AAP,1.13
2,5434,2020-10-01 04:00:00,153.56,...,938944,AAP,0.90
3,5435,2020-10-02 04:00:00,152.24,...,785520,AAP,0.47
4,5436,2020-10-05 04:00:00,155.74,...,951671,AAP,0.15
...,...,...,...,...,...,...,...
21446,6860,2020-10-22 04:00:00,30.11,...,159568,ZUMZ,3.00
21447,6861,2020-10-23 04:00:00,31.24,...,147693,ZUMZ,0.61
21448,6862,2020-10-26 04:00:00,30.86,...,141009,ZUMZ,-1.32
21449,6863,2020-10-27 04:00:00,30.51,...,95465,ZUMZ,-2.15


# Para ir trabajando

* Finviz: wrapper de datos basicos 


* Otros paneles
    * Precios en vivo paneles de FMP


* Historicos
    * Equity historico con batch requests ALPACA
    * Equity con yahoo
    * Buscar alternativas como investing


* Intradiarios    
    * Intradiario IEX / Alpaca


* Cadenas de Opciones
    * Ameritrade precios/puntas/Griegas
    * Yahoo